In [7]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
#Import Dependencies
import pandas as pd
import requests
from config import napster_key, CLIENT_ID, CLIENT_SECRET

In [2]:
#Napster API Base
napster_base = 'https://api.napster.com/v2.2'
header = {'apikey': napster_key}

In [3]:
#Retrieve Genres From Napster
genres_english = '/genres?lang=en-EN'
genre_url = f'{napster_base}{genres_english}'
response = requests.get(genre_url, headers=header).json()

In [4]:
#Create Lists to Hold Genres and Their Napster IDs
genre_list = [] 
id_list = []

#Loop Through API Response to Add Details to Lists
for x in range(0,23):
    genre = response['genres'][x]['name']
    genre_list.append(genre)
    id = response['genres'][x]['id']
    id_list.append(id)

In [10]:
#Create an Empty DataFrame for Genre and Artist Info
genre_artist_df = pd.DataFrame(columns = ['Genre', 'Genre ID', 'Artist'])

In [12]:
#Request the Top 20 Artists for each Genre
for x in range(0,23):
    id = id_list[x]
    genre = genre_list[x]
    top_artists = f'/genres/{id}/artists/top'
    limit = '?limit=20'
    timeframe= '&range=year'
    url = f'{napster_base}{top_artists}{limit}{timeframe}'
    response = requests.get(url, headers=header).json()

    #Create a DataFrame of Genres and Artist from the Responses
    for y in range(0,20):
        artist = response['artists'][y]['name']
        genre_artist_df = genre_artist_df.append({'Genre': genre, 'Genre ID':id, 'Artist':artist}, ignore_index=True)

#Check the Length of the Completed DataFrame
len(genre_artist_df)

460

In [13]:
#Preview the DataFrame
genre_artist_df.head(1)

,Genre,Genre ID,Artist
0,Pop,g.115,Michael Jackson


In [14]:
#Spotify API 
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [15]:
# base URL of all Spotify API endpoints
Search_URL = 'https://api.spotify.com/v1/search'

In [16]:
#Test Request to Locate Spotify URI in Response
artist_name= "Linkin Park"

artist_info = requests.get(
Search_URL,
headers=headers,
params={'q': artist_name, 'type': 'artist'}).json()

uri = artist_info['artists']['items'][0]['id']
print(uri)

6XyY86QOPPrYVGvF9ch6wz


In [17]:
#Create a List to hold Spotify Artist IDs
id = []

for artist in genre_artist_df['Artist']:
    try:
        artist_info = requests.get(
        Search_URL,
        headers=headers,
        params={'q': artist, 'type': 'artist'}).json()

        uri = artist_info['artists']['items'][0]['id']
        id.append(uri)
    except:
        print(f'{artist} not found on Spotify!')
        id.append('NaN')

#Check that Length of the ID List
len(id)

Diplo feat. Lil Pump, Juicy J, Famous Dex & French Montana not found on Spotify!


460

In [18]:
#Create a New Column for the Artist Spotify ID
genre_artist_df['Spotify ID'] = id
genre_artist_df.head(1)

,Genre,Genre ID,Artist,Spotify ID
0,Pop,g.115,Michael Jackson,3fMbdgg4jU18AjLCKBhRSm


In [19]:
#Import the Spotipy Library
import spotipy
from spotipy.oauth2 import SpotifyOAuth

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                                               client_secret=CLIENT_SECRET,
                                               redirect_uri='http://localhost:1234',
                                               scope="user-library-read"))

In [20]:
#Test Spotify ID to Locate Track information using Spotipy
mj_id = 'spotify:artist:3fMbdgg4jU18AjLCKBhRSm'
tracks = sp.artist_top_tracks(mj_id,country='US')
print(tracks['tracks'][0]['album']['artists'][0]['name'])

Michael Jackson


In [22]:
#Review How Spotipy Returns Track URIs
for track in tracks['tracks'][:5]:
    print('track    : ' + track['name'] + '| uri: ' + track['uri'])

track    : Billie Jean| uri: spotify:track:7J1uxwnxfQLu4APicE5Rnj
track    : Beat It| uri: spotify:track:3BovdzfaX4jb5KFQwoPfAw
track    : Chicago| uri: spotify:track:5BKKy9fIJL5uM9fz1SnqyP
track    : Smooth Criminal - 2012 Remaster| uri: spotify:track:5T7ywazdGIydr6JCW6t02j
track    : Rock with You - Single Version| uri: spotify:track:7oOOI85fVQvVnK5ynNMdW7


In [23]:
#Test Spotipy Audio Features Method
track1 = sp.audio_features('spotify:track:2bCQHF9gdG5BNDVuEIEnNk')
print(track1)
type(track1[0])

[{'danceability': 0.853, 'energy': 0.981, 'key': 8, 'loudness': -3.947, 'mode': 1, 'speechiness': 0.0751, 'acousticness': 0.247, 'instrumentalness': 0.468, 'liveness': 0.306, 'valence': 0.595, 'tempo': 118.193, 'type': 'audio_features', 'id': '2bCQHF9gdG5BNDVuEIEnNk', 'uri': 'spotify:track:2bCQHF9gdG5BNDVuEIEnNk', 'track_href': 'https://api.spotify.com/v1/tracks/2bCQHF9gdG5BNDVuEIEnNk', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2bCQHF9gdG5BNDVuEIEnNk', 'duration_ms': 257760, 'time_signature': 4}]


dict

In [24]:
#Find Index of Artist Error
genre_artist_df[genre_artist_df['Artist'] == 'Diplo feat. Lil Pump, Juicy J, Famous Dex & French Montana'].index

Int64Index([383], dtype='int64')

In [25]:
#Remove Error From DataFrame
cleaned_artist_df =genre_artist_df.drop(genre_artist_df.index[383], axis=0)
len(cleaned_artist_df)

459

In [26]:
#Create a Dictionary as a Placeholder for Errors in API Request
error_dict = {'danceability': 'NA', 'energy': 'NA', 'key': 'NA', 'loudness': 'NA', 'mode': 'NA',
 'speechiness': 'NA', 'acousticness': 'NA', 'instrumentalness': 'NA', 'liveness': 'NA', 
 'valence': 'NA', 'tempo': 'NA', 'type': 'audio_features', 'id': 'NA', 
 'uri': 'NA', 'track_href': 'NA', 
 'analysis_url': 'NA', 'duration_ms': 'NA', 'time_signature': 'NA'}

In [27]:
#Create a List to Hold Track Features
track_features = []

#Nested API Calls to Get the Top 10 Tracks For Each Artist 
#& Then The Track Features For Each Track
#If Any Errors, Print Artist Name, Return Artist Info and The Error Dictionary as Placeholder
for s_id in cleaned_artist_df['Spotify ID']:
    current_id = f'spotify:artist:{s_id}'
    current_tracks = sp.artist_top_tracks(current_id,country='US')
    try:
        for track in current_tracks['tracks'][:10]:
            track_info = {'Artist': track['album']['artists'][0]['name'], 'Track': track['name'], 'Track ID': track['uri']}
            track_audiofeat = sp.audio_features(track['uri'])
            track_info.update(track_audiofeat[0])
            track_features.append(track_info)
    except:
        name = track['name']
        print(f'{name} encountered an error!')
        track_info = {'Artist': track['album']['artists'][0]['name'], 'Track': track['name'], 'Track ID': track['uri']}
        track_info.update(error_dict)
        track_features.append(track_info)

50 Reggae Lovers Rock Songs - Continuous Mix encountered an error!


In [28]:
#Create a DataFrame From The Track Features List of Dictionaries
track_feat_df = pd.DataFrame(track_features)
track_feat_df.head(1)

,Artist,Track,Track ID,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Michael Jackson,Billie Jean,spotify:track:7J1uxwnxfQLu4APicE5Rnj,0.932,0.457,11,-9.62,0,0.0541,0.0173,...,0.0414,0.884,117.002,audio_features,7J1uxwnxfQLu4APicE5Rnj,spotify:track:7J1uxwnxfQLu4APicE5Rnj,https://api.spotify.com/v1/tracks/7J1uxwnxfQLu...,https://api.spotify.com/v1/audio-analysis/7J1u...,294227,4


In [29]:
#Find Index of Track Error
track_feat_df[track_feat_df['Track'] == '50 Reggae Lovers Rock Songs - Continuous Mix'].index

Int64Index([2350], dtype='int64')

In [30]:
#Remove Track Error From DataFrame
track_feat_df.drop(track_feat_df.index[2350], axis=0,inplace=True)
len(cleaned_artist_df)

459

In [32]:
#Combine Track Features DataFrame with Genre DataFrame
merged_genre_tracks = pd.merge(track_feat_df,genre_artist_df,how='left', on='Artist')
merged_genre_tracks.head(1)

,Artist,Track,Track ID,danceability,energy,key,loudness,mode,speechiness,acousticness,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,Genre,Genre ID,Spotify ID
0,Michael Jackson,Billie Jean,spotify:track:7J1uxwnxfQLu4APicE5Rnj,0.932,0.457,11,-9.62,0,0.0541,0.0173,...,audio_features,7J1uxwnxfQLu4APicE5Rnj,spotify:track:7J1uxwnxfQLu4APicE5Rnj,https://api.spotify.com/v1/tracks/7J1uxwnxfQLu...,https://api.spotify.com/v1/audio-analysis/7J1u...,294227,4,Pop,g.115,3fMbdgg4jU18AjLCKBhRSm


In [33]:
#Save Final DataSet to CSV File to be used for Analysis
merged_genre_tracks.to_csv('../Music-Research-Project-1/Resources/api_dataset.csv')